<a href="https://colab.research.google.com/github/berniwal/CIL_Project/blob/master/CIL-Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Go to correct directory

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.getcwd()
os.chdir('/content/drive/My Drive/CIL')
os.getcwd()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


'/content/drive/My Drive/CIL'

## install + import libraries

In [2]:
!pip install params_flow==0.7.1
!pip install py-params==0.7.3
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

import tensorflow as tf
from tensorflow import keras
import pandas as pd
import os
from datetime import datetime

import bert
from bert.tokenization.bert_tokenization import FullTokenizer
from bert.loader import StockBertConfig, map_stock_config_to_params, load_stock_weights
from bert import BertModelLayer

from tqdm import tqdm
import numpy as np

from sklearn.utils import shuffle

  Created wheel for params-flow: filename=params_flow-0.7.1-cp36-none-any.whl size=15376 sha256=f5b99bec221ca97b017a6f3375326aa458d1acd40b398593172bf87b9148a822
  Stored in directory: /root/.cache/pip/wheels/e2/7b/2a/b411aaa219132a68b17937fc9431fd9eb9c23c12a7df3d134f
  Created wheel for py-params: filename=py_params-0.9.7-cp36-none-any.whl size=7302 sha256=d67c8000c33e21a27e89e852382971baed9ea544cb0a68525b5c24e7ec3d5172
  Stored in directory: /root/.cache/pip/wheels/67/f5/19/b461849a50aefdf4bab47c4756596e82ee2118b8278e5a1980
Successfully built params-flow py-params
  Created wheel for py-params: filename=py_params-0.7.3-cp36-none-any.whl size=4346 sha256=3b07f84cbe405b38116e877540d7580e80a5ed0711a34abb785505cb5727559f
  Stored in directory: /root/.cache/pip/wheels/a3/9f/a1/e7e79bd92eecef952a46b16d0bc93ffdc91d4b619f79777d27
Successfully built py-params
  Found existing installation: py-params 0.9.7
    Uninstalling py-params-0.9.7:
      Successfully uninstalled py-params-0.9.7


## Fix Directories for BERT
First part of selecting which model

In [0]:
EXPERIMENT = 'first_experiment'

CHECKPOINT_DIR = './checkpoints'

CHECKPOINT = './bert/checkpoints/bert_base'
#CHECKPOINT = './bert/checkpoints/bert_large_wwm'

CHECKPOINT_CKPT = os.path.join(CHECKPOINT, 'bert_model.ckpt')
CHECKPOINT_VOCAB = os.path.join(CHECKPOINT, 'vocab.txt')
CHECKPOINT_CONFIG = os.path.join(CHECKPOINT, 'bert_config.json')

DATASET_DIR = './'

DATASET_FILE_TRAIN_NEG = os.path.join(DATASET_DIR, 'twitter-datasets/train_neg_full.txt')
DATASET_FILE_TRAIN_POS = os.path.join(DATASET_DIR, 'twitter-datasets/train_pos_full.txt')
DATASET_FILE_TEST = os.path.join(DATASET_DIR, 'twitter-datasets/test_data.txt')

FILE_PATHS = [DATASET_FILE_TRAIN_POS, DATASET_FILE_TRAIN_NEG]

#402, 324
MAX_SEQ_LENGTH = 128
LEARNING_RATE = 2e-5
BATCH_SIZE = 16
NUM_TRAIN_EPOCHS = 3.0
WARMUP_PROPORTION = 0.1
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

label_list = [0, 1]

## methods to create BERT layer
change name="albert" for other model, have to adjust paths above too

In [0]:
def flatten_layers(root_layer):
    if isinstance(root_layer, keras.layers.Layer):
        yield root_layer
    for layer in root_layer._layers:
        for sub_layer in flatten_layers(layer):
            yield sub_layer


def freeze_bert_layers(l_bert):
    """
    Freezes all but LayerNorm and adapter layers - see arXiv:1902.00751.
    """
    for layer in flatten_layers(l_bert):
        if layer.name in ["LayerNorm", "adapter-down", "adapter-up"]:
            layer.trainable = True
        elif len(layer._layers) == 0:
            layer.trainable = False
        l_bert.embeddings_layer.trainable = False


def create_learning_rate_scheduler(max_learn_rate=5e-5,
                                   end_learn_rate=1e-7,
                                   warmup_epoch_count=10,
                                   total_epoch_count=90):

    def lr_scheduler(epoch):
        if epoch < warmup_epoch_count:
            #res = (max_learn_rate/warmup_epoch_count) * (epoch + 1)
            res = end_learn_rate
        else:
            res = max_learn_rate*math.exp(math.log(end_learn_rate/max_learn_rate)*(epoch-warmup_epoch_count+1)/(total_epoch_count-warmup_epoch_count+1))
        return float(res)
    learning_rate_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_scheduler, verbose=1)

    return learning_rate_scheduler


def create_model(max_seq_len, adapter_size=64):
    """Creates a classification model."""

    # adapter_size = 64  # see - arXiv:1902.00751

    # create the bert layer
    with tf.io.gfile.GFile(CHECKPOINT_CONFIG, "r") as reader:
        bc = StockBertConfig.from_json_string(reader.read())
        bert_params = map_stock_config_to_params(bc)
        bert_params.adapter_size = adapter_size
        bert = BertModelLayer.from_params(bert_params, name="bert")

    input_ids = keras.layers.Input(shape=(max_seq_len,), dtype='int32', name="input_ids")
    # token_type_ids = keras.layers.Input(shape=(max_seq_len,), dtype='int32', name="token_type_ids")
    # output         = bert([input_ids, token_type_ids])
    output = bert(input_ids)

    print("bert shape", output.shape)
    cls_out = keras.layers.Lambda(lambda seq: seq[:, 0, :])(output)
    cls_out = keras.layers.Dropout(0.5)(cls_out)
    logits = keras.layers.Dense(units=768, activation="tanh")(cls_out)
    logits = keras.layers.Dropout(0.5)(logits)
    logits = keras.layers.Dense(units=2, activation="softmax")(logits)

    # model = keras.Model(inputs=[input_ids, token_type_ids], outputs=logits)
    # model.build(input_shape=[(None, max_seq_len), (None, max_seq_len)])
    model = keras.Model(inputs=input_ids, outputs=logits)
    model.build(input_shape=(None, max_seq_len))

    # load the pre-trained model weights
    load_stock_weights(bert, CHECKPOINT_CKPT)

    # freeze weights if adapter-BERT is used
    if adapter_size is not None:
        freeze_bert_layers(bert)

    model.compile(optimizer=keras.optimizers.Adam(),
                  loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")])

    return model

## BERT text preprocessing

In [0]:
TOKENIZER = FullTokenizer(vocab_file=CHECKPOINT_VOCAB, do_lower_case=True)

def pre_process_positive(x, max_seq_len):
  tokens = TOKENIZER.tokenize(x.numpy())
  tokens = ["[CLS]"] + tokens + ["[SEP]"]
  token_ids = TOKENIZER.convert_tokens_to_ids(tokens)
  token_ids = token_ids[:min(len(token_ids), max_seq_len - 2)]
  token_ids = np.concatenate((token_ids, np.zeros((max_seq_len - len(token_ids))))).astype(np.int32)
  return token_ids, int(1)

def pre_process_negative(x, max_seq_len):
  tokens = TOKENIZER.tokenize(x.numpy())
  tokens = ["[CLS]"] + tokens + ["[SEP]"]
  token_ids = TOKENIZER.convert_tokens_to_ids(tokens)
  token_ids = token_ids[:min(len(token_ids), max_seq_len - 2)]
  token_ids = np.concatenate((token_ids, np.zeros((max_seq_len - len(token_ids))))).astype(np.int32)
  return token_ids, int(0)

def pre_process_text(x):
  tokens = TOKENIZER.tokenize(x.numpy())
  tokens = ["[CLS]"] + tokens + ["[SEP]"]
  return tokens

def dummy_pre_process(x):
   return tf.constant(3, shape=(128,))

In [8]:
'''positive = tf.data.TextLineDataset(DATASET_FILE_TRAIN_POS)

f = open("./twitter-datasets/preprocessed_positive.txt", "w")

import tensorflow_datasets as tfds

ENCODER = tfds.features.text.SubwordTextEncoder(
    vocab_list=TOKENIZER.vocab
)

for x in positive:
  write = pre_process_text(x)
  write = ''.join(map(str, write))
  f.write(write + '\n')

f.close()'''

'positive = tf.data.TextLineDataset(DATASET_FILE_TRAIN_POS)\n\nf = open("./twitter-datasets/preprocessed_positive.txt", "w")\n\nimport tensorflow_datasets as tfds\n\nENCODER = tfds.features.text.SubwordTextEncoder(\n    vocab_list=TOKENIZER.vocab\n)\n\nfor x in positive:\n  write = pre_process_text(x)\n  write = \'\'.join(map(str, write))\n  f.write(write + \'\n\')\n\nf.close()'

In [9]:
'''positive = tf.data.TextLineDataset(DATASET_FILE_TRAIN_POS)
negative = tf.data.TextLineDataset(DATASET_FILE_TRAIN_NEG)

positive = positive.map(
    lambda x: tf.py_function(pre_process_positive, [x,128], (tf.int32, tf.int32)),
    num_parallel_calls=tf.data.experimental.AUTOTUNE)
negative = negative.map(
    lambda x: tf.py_function(pre_process_negative, [x, 128], (tf.int32, tf.int32)),
    num_parallel_calls=tf.data.experimental.AUTOTUNE)

dataset = tf.data.experimental.sample_from_datasets([positive,negative]).batch(32)'''

'positive = tf.data.TextLineDataset(DATASET_FILE_TRAIN_POS)\nnegative = tf.data.TextLineDataset(DATASET_FILE_TRAIN_NEG)\n\npositive = positive.map(\n    lambda x: tf.py_function(pre_process_positive, [x,128], (tf.int32, tf.int32)),\n    num_parallel_calls=tf.data.experimental.AUTOTUNE)\nnegative = negative.map(\n    lambda x: tf.py_function(pre_process_negative, [x, 128], (tf.int32, tf.int32)),\n    num_parallel_calls=tf.data.experimental.AUTOTUNE)\n\ndataset = tf.data.experimental.sample_from_datasets([positive,negative]).batch(32)'

# Store Data in TF-Record Files, such that we don't get Memory Issues and have a valid Input Pipeline.

In [10]:
'''positive = tf.data.TextLineDataset(DATASET_FILE_TRAIN_POS)
negative = tf.data.TextLineDataset(DATASET_FILE_TRAIN_NEG)
file_name = './twitter-datasets/tfrecord_files/positive.tfrecord'

with tf.io.TFRecordWriter(file_name) as writer: 
  for idx, x in tqdm(enumerate(positive)):
    sentence, label = pre_process_positive(x, 128)

    feature = {
        "sentence": tf.train.Feature(int64_list=tf.train.Int64List(value=sentence)),
        "label": tf.train.Feature(int64_list=tf.train.Int64List(value=[label]))
    }

    proto = tf.train.Example(features=tf.train.Features(feature=feature))
    sample = proto.SerializeToString()

    writer.write(sample)

file_name_negative = './twitter-datasets/tfrecord_files/negative.tfrecord'

with tf.io.TFRecordWriter(file_name_negative) as writer: 
  for idx, x in tqdm(enumerate(negative)):
    sentence, label = pre_process_negative(x, 128)

    feature = {
        "sentence": tf.train.Feature(int64_list=tf.train.Int64List(value=sentence)),
        "label": tf.train.Feature(int64_list=tf.train.Int64List(value=[label]))
    }

    proto = tf.train.Example(features=tf.train.Features(feature=feature))
    sample = proto.SerializeToString()

    writer.write(sample)'''

'positive = tf.data.TextLineDataset(DATASET_FILE_TRAIN_POS)\nnegative = tf.data.TextLineDataset(DATASET_FILE_TRAIN_NEG)\nfile_name = \'./twitter-datasets/tfrecord_files/positive.tfrecord\'\n\nwith tf.io.TFRecordWriter(file_name) as writer: \n  for idx, x in tqdm(enumerate(positive)):\n    sentence, label = pre_process_positive(x, 128)\n\n    feature = {\n        "sentence": tf.train.Feature(int64_list=tf.train.Int64List(value=sentence)),\n        "label": tf.train.Feature(int64_list=tf.train.Int64List(value=[label]))\n    }\n\n    proto = tf.train.Example(features=tf.train.Features(feature=feature))\n    sample = proto.SerializeToString()\n\n    writer.write(sample)\n\nfile_name_negative = \'./twitter-datasets/tfrecord_files/negative.tfrecord\'\n\nwith tf.io.TFRecordWriter(file_name_negative) as writer: \n  for idx, x in tqdm(enumerate(negative)):\n    sentence, label = pre_process_negative(x, 128)\n\n    feature = {\n        "sentence": tf.train.Feature(int64_list=tf.train.Int64List(v

# First 1 Mio
Also loading data into array


In [11]:
def load_data(file_path):
    data = {}
    data["sentence"] = []
    with open(file_path, "r") as f:
        data["sentence"] = f.readlines()

    #longest_string = max(data["sentence"], key=len)
    #print(longest_string)
    #print(len(longest_string))

    return pd.DataFrame.from_dict(data)

def load_dataset(pos_directory, neg_directory):
    pos_df = load_data(pos_directory)
    neg_df = load_data(neg_directory)

    pos_df["sentiment"] = 1
    neg_df["sentiment"] = 0

    return pd.concat([pos_df, neg_df])

class MovieReviewData:
    DATA_COLUMN = "sentence"
    LABEL_COLUMN = "sentiment"

    def __init__(self, tokenizer= FullTokenizer, sample_size=None, max_seq_len=1024):
        self.tokenizer = tokenizer
        self.sample_size = sample_size
        self.max_seq_len = 0
        trainset = load_dataset(DATASET_FILE_TRAIN_POS, DATASET_FILE_TRAIN_NEG)

        trainset = shuffle(trainset, random_state=5)

        train = trainset.head(2250000)
        test = trainset.tail(250000)

        train = shuffle(train)
        test = shuffle(test)

        trainset.reset_index(inplace=True, drop=True)

        #train, test = map(lambda df: df.reindex(df[MovieReviewData.DATA_COLUMN].str.len().sort_values().index),
        #                 [train, test])

        if sample_size is not None:
            assert sample_size % 128 == 0
            train, test = train.head(sample_size), test.head(250000)
            # train, test = map(lambda df: df.sample(sample_size), [train, test])

        ((self.train_x, self.train_y),
         (self.test_x, self.test_y)) = map(self._prepare, [train, test])

        print("max seq_len", self.max_seq_len)
        self.max_seq_len = max_seq_len
        #self.max_seq_len = min(self.max_seq_len, max_seq_len)
        ((self.train_x, self.train_x_token_types),
         (self.test_x, self.test_x_token_types)) = map(self._pad,
                                                       [self.train_x, self.test_x])

    def _prepare(self, df):
        x, y = [], []
        with tqdm(total=df.shape[0], unit_scale=True) as pbar:
            for ndx, row in df.iterrows():
                text, label = row[MovieReviewData.DATA_COLUMN], row[MovieReviewData.LABEL_COLUMN]
                tokens = self.tokenizer.tokenize(text)
                tokens = ["[CLS]"] + tokens + ["[SEP]"]
                token_ids = self.tokenizer.convert_tokens_to_ids(tokens)
                self.max_seq_len = max(self.max_seq_len, len(token_ids))
                x.append(token_ids)
                y.append(int(label))
                pbar.update()
        return np.array(x), np.array(y)

    def _pad(self, ids):
        x, t = [], []
        token_type_ids = [0] * self.max_seq_len
        for input_ids in ids:
            input_ids = input_ids[:min(len(input_ids), self.max_seq_len - 2)]
            input_ids = input_ids + [0] * (self.max_seq_len - len(input_ids))
            x.append(np.array(input_ids))
            t.append(token_type_ids)
        return np.array(x), np.array(t)

tokenizer = FullTokenizer(vocab_file=CHECKPOINT_VOCAB, do_lower_case=True)
data = MovieReviewData(tokenizer, sample_size=10*128*878, max_seq_len=128)

100%|██████████| 1.12M/1.12M [08:00<00:00, 2.34kit/s]
100%|██████████| 250k/250k [01:45<00:00, 2.36kit/s]


max seq_len 113


# Second Mio (not needed for first tests)


In [12]:
'''
def load_data(file_path):
    data = {}
    data["sentence"] = []
    with open(file_path, "r") as f:
        data["sentence"] = f.readlines()

    #longest_string = max(data["sentence"], key=len)
    #print(longest_string)
    #print(len(longest_string))

    return pd.DataFrame.from_dict(data)

def load_dataset(pos_directory, neg_directory):
    pos_df = load_data(pos_directory)
    neg_df = load_data(neg_directory)

    pos_df["sentiment"] = 1
    neg_df["sentiment"] = 0

    return pd.concat([pos_df, neg_df])

class MovieReviewData:
    DATA_COLUMN = "sentence"
    LABEL_COLUMN = "sentiment"

    def __init__(self, tokenizer= FullTokenizer, sample_size=None, max_seq_len=1024):
        self.tokenizer = tokenizer
        self.sample_size = sample_size
        self.max_seq_len = 0
        trainset = load_dataset(DATASET_FILE_TRAIN_POS, DATASET_FILE_TRAIN_NEG)

        trainset = shuffle(trainset, random_state=5)

        train = trainset.tail(1375000)
        train = train.head(1125000)
        test = trainset.tail(250000)

        train = shuffle(train)
        test = shuffle(test)

        trainset.reset_index(inplace=True, drop=True)

        #train, test = map(lambda df: df.reindex(df[MovieReviewData.DATA_COLUMN].str.len().sort_values().index),
        #                 [train, test])

        if sample_size is not None:
            assert sample_size % 128 == 0
            train, test = train.head(sample_size), test.head(250000)
            # train, test = map(lambda df: df.sample(sample_size), [train, test])

        ((self.train_x, self.train_y),
         (self.test_x, self.test_y)) = map(self._prepare, [train, test])

        print("max seq_len", self.max_seq_len)
        self.max_seq_len = max_seq_len
        #self.max_seq_len = min(self.max_seq_len, max_seq_len)
        ((self.train_x, self.train_x_token_types),
         (self.test_x, self.test_x_token_types)) = map(self._pad,
                                                       [self.train_x, self.test_x])

    def _prepare(self, df):
        x, y = [], []
        with tqdm(total=df.shape[0], unit_scale=True) as pbar:
            for ndx, row in df.iterrows():
                text, label = row[MovieReviewData.DATA_COLUMN], row[MovieReviewData.LABEL_COLUMN]
                tokens = self.tokenizer.tokenize(text)
                tokens = ["[CLS]"] + tokens + ["[SEP]"]
                token_ids = self.tokenizer.convert_tokens_to_ids(tokens)
                self.max_seq_len = max(self.max_seq_len, len(token_ids))
                x.append(token_ids)
                y.append(int(label))
                pbar.update()
        return np.array(x), np.array(y)

    def _pad(self, ids):
        x, t = [], []
        token_type_ids = [0] * self.max_seq_len
        for input_ids in ids:
            input_ids = input_ids[:min(len(input_ids), self.max_seq_len - 2)]
            input_ids = input_ids + [0] * (self.max_seq_len - len(input_ids))
            x.append(np.array(input_ids))
            t.append(token_type_ids)
        return np.array(x), np.array(t)

tokenizer = FullTokenizer(vocab_file=CHECKPOINT_VOCAB, do_lower_case=True)
data = MovieReviewData(tokenizer, sample_size=10*128*878, max_seq_len=128)
'''

100%|██████████| 1.12M/1.12M [07:50<00:00, 2.39kit/s]
100%|██████████| 250k/250k [01:44<00:00, 2.40kit/s]


max seq_len 116


In [13]:
'''def tfrecord_to_dataset(filenames):
  ds = tf.data.TFRecordDataset(filenames)
  feature_description = {
      "sentence":  tf.io.VarLenFeature(tf.int64),
      "label":      tf.io.FixedLenFeature([], tf.int64, default_value=-1)
  }

  def parse_proto(proto):
    example = tf.io.parse_single_example(proto, feature_description)
    token_ids, label = example["sentence"], example["label"]
    token_ids = tf.sparse.to_dense(token_ids)
    return token_ids, label

  return ds.map(parse_proto)'''

'def tfrecord_to_dataset(filenames):\n  ds = tf.data.TFRecordDataset(filenames)\n  feature_description = {\n      "sentence":  tf.io.VarLenFeature(tf.int64),\n      "label":      tf.io.FixedLenFeature([], tf.int64, default_value=-1)\n  }\n\n  def parse_proto(proto):\n    example = tf.io.parse_single_example(proto, feature_description)\n    token_ids, label = example["sentence"], example["label"]\n    token_ids = tf.sparse.to_dense(token_ids)\n    return token_ids, label\n\n  return ds.map(parse_proto)'

In [14]:
'''file_name = './twitter-datasets/tfrecord_files/positive.tfrecord'
file_name_negative = './twitter-datasets/tfrecord_files/negative.tfrecord'

ds = tfrecord_to_dataset([file_name, file_name_negative])
ds = ds.shuffle(buffer_size=2500000, seed=1000)
ds = ds.batch(32, drop_remainder = True)

for x in ds.take(1):
  print(x)'''

"file_name = './twitter-datasets/tfrecord_files/positive.tfrecord'\nfile_name_negative = './twitter-datasets/tfrecord_files/negative.tfrecord'\n\nds = tfrecord_to_dataset([file_name, file_name_negative])\nds = ds.shuffle(buffer_size=2500000, seed=1000)\nds = ds.batch(32, drop_remainder = True)\n\nfor x in ds.take(1):\n  print(x)"

## data from array to tensor

In [0]:
dataset_train = tf.data.Dataset.from_tensor_slices((data.train_x, data.train_y))
dataset_train = dataset_train.batch(32, drop_remainder=True)

dataset_test = tf.data.Dataset.from_tensor_slices((data.test_x, data.test_y))
dataset_test = dataset_test.batch(32, drop_remainder=True)

In [0]:
## Build model

In [16]:
max_seq_len = 128
adapter_size = None

with tpu_strategy.scope():
    model = create_model(max_seq_len, adapter_size=adapter_size)

#model = create_model(max_seq_len, adapter_size=adapter_size)

#model.load_weights("twitter_long_two.h5")

model.summary()

total_epoch_count = 1

bert shape (None, 128, 768)
Done loading 196 BERT weights from: ./bert/checkpoints/bert_base/bert_model.ckpt into <bert.model.BertModelLayer object at 0x7f8f07fc3c88> (prefix:bert). Count of weights not found in the checkpoint was: [0]. Count of weights with mismatched shape: [0]
Unused weights from checkpoint: 
	bert/embeddings/token_type_embeddings
	bert/pooler/dense/bias
	bert/pooler/dense/kernel
	cls/predictions/output_bias
	cls/predictions/transform/LayerNorm/beta
	cls/predictions/transform/LayerNorm/gamma
	cls/predictions/transform/dense/bias
	cls/predictions/transform/dense/kernel
	cls/seq_relationship/output_bias
	cls/seq_relationship/output_weights
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ids (InputLayer)       [(None, 128)]             0         
_________________________________________________________________
bert (BertModelLayer)        (None, 128, 768)          

## Currently no model to load

In [17]:
#model.load_weights("twitter_bert_large_second.h5")

OSError: ignored

## Train model/fit+save weights

In [18]:
model.fit(dataset_train,
          validation_data=dataset_test,
          epochs=total_epoch_count,
          callbacks=[create_learning_rate_scheduler(max_learn_rate=1e-5,
                                                  end_learn_rate=1e-7,
                                                  warmup_epoch_count=3,
                                                  total_epoch_count=total_epoch_count),
                    keras.callbacks.EarlyStopping(patience=20, restore_best_weights=True),
                  ])


model.save_weights('./twitter_bert_large_third.h5', overwrite=True)


Epoch 00001: LearningRateScheduler reducing learning rate to 1e-07.
35120/35120 [==============================] - 3016s 86ms/step - loss: 0.6376 - acc: 0.6416 - val_loss: 0.5918 - val_acc: 0.7027 - lr: 1.0000e-07


## ?

In [19]:
'''
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()

# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
  process = psutil.Process(os.getpid())
  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))

printm()
'''

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7413 sha256=20b71cdb5471a77101852c3e4be50b2da27e3a5f7c0cd2e591ad0a430fa28f14
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil


IndexError: ignored

In [20]:
!nvidia-smi

/bin/bash: nvidia-smi: command not found


## Preprocess test-set

In [21]:
positive_data = []
with open(DATASET_FILE_TRAIN_POS, "r") as f:
  for x in f:
    positive_data.append(x)

negative_data = []
with open(DATASET_FILE_TRAIN_NEG, "r") as f:
  for x in f:
    negative_data.append(x)


print(len(positive_data))
positive_data = list(dict.fromkeys(positive_data))
print(len(positive_data))

print(len(negative_data))
negative_data = list(dict.fromkeys(negative_data))
print(len(negative_data))
test_data = []
with open(DATASET_FILE_TEST, "r") as f:
  for x in f:
    test_data.append(x)

encoded_test_data = []
for x in test_data:
  result, _ = pre_process_positive(tf.convert_to_tensor(x), 128)
  encoded_test_data.append(tf.reshape(tf.convert_to_tensor(result), (1,128)))
encoded_test_ds = tf.data.Dataset.from_tensor_slices(encoded_test_data)
for x in encoded_test_ds.take(5):
  print(x)

1250000
1127644
1250000
1142838


## Predict test set

In [23]:
results = model.predict(encoded_test_ds)
labels = np.argmax(results, axis=1)

[[9.9995625e-01 4.3788335e-05]
 [4.6669193e-02 9.5333087e-01]
 [8.9574224e-01 1.0425783e-01]
 ...
 [9.9997592e-01 2.4031357e-05]
 [1.1560619e-03 9.9884397e-01]
 [9.9957913e-01 4.2092413e-04]]
[0 1 0 ... 0 1 0]


## Postprocessing

In [0]:
#labels = [1 if x == 1 else -1 for x in labels]

#reverse labels
labels = [-1 if x == 1 else 1 for x in labels]
#un-reverse labels
labels = [1 if x == -1 else -1 for x in labels]

In [26]:
df = pd.DataFrame({'Id': np.arange(1,10001),
                  'Prediction': labels
                   })
df.to_csv('bert_large_1point5.csv', index=False)

[-1, 1, -1, 1, -1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, -1, 1, -1, 1, -1, 1, -1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, -1, -1, -1, -1, -1, -1, 1, 1, -1, 1, 1, 1, -1, 1, 1, 1, 1, 1, -1, 1, 1, 1, -1, 1, 1, 1, 1, -1, -1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, -1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, -1, 1, -1, 1, -1, -1, -1, -1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, -1, 1, -1, -1, -1, -1, 1, 1, 1, 1, 1, 1, -1, 1, -1, 1, -1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, -1, 1, 1, 1, -1, 1, 1, -1, 1, -1, -1, -1, -1, -1, -1, -1, 1, 1, -1, 1, -1, 1, -1, 1, 1, 1, 1, 1, 1, -1, -1, 1, 1, 1, 1, -1, 1, 1, -1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, -1, -1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, 1, 1, -1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, -1, 1, 1, 1, -1, 1, -1, 1, 1, -1, -1, 1, 1, 1, -1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, -1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, -1, 